In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers

Using TensorFlow backend.
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hsiaoyuhsien/opt

In [3]:
import Image_Preprocess
BCC_train=Image_Preprocess.BCC_train()
BCC_test=Image_Preprocess.BCC_test()
FCC_train=Image_Preprocess.FCC_train()
FCC_test=Image_Preprocess.FCC_test()

In [4]:
FCC_test=np.array(FCC_test)
BCC_test=np.array(BCC_test)
FCC_train=np.array(FCC_train)
BCC_train=np.array(BCC_train)


In [6]:
# making training data and test data

train_data=np.append(FCC_train,BCC_train,axis=0)
test_data=np.append(FCC_test,BCC_test,axis=0)

# making train_label
train_label=[]
for i in range(119):
    train_label.append('FCC')
for i in range(119):
    train_label.append('BCC')
    
test_label=[]
for i in range(9):
    test_label.append('FCC')
for i in range(9):
    test_label.append('BCC')
    
train_label=np.array(train_label)
test_label=np.array(test_label)


In [7]:
# reshaping data
train_data = train_data.reshape(-1,288,432,1).astype('float32')/255
test_data = test_data.reshape(-1,288,432,1).astype('float32')/255

In [8]:
train_data.shape

(238, 288, 432, 1)

In [9]:
# one hot encoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_label_10 = label_encoder.fit_transform(train_label)
test_label_10 = label_encoder.fit_transform(test_label)
train_label = to_categorical(train_label_10)
test_label = to_categorical(test_label_10)

In [10]:
# cross validation
train_data,val_data,train_label,val_label=train_test_split(train_data,train_label,test_size=0.25,random_state=13)

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import TensorBoard

model=Sequential([
    Convolution2D(32,3,3,input_shape=(288,432,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Convolution2D(64,3,3,input_shape=(288,432,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Convolution2D(64,3,3,input_shape=(288,432,1),activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()




/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(288, 432,..., activation="relu")`
  
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(288, 432,..., activation="relu")`
  
/Users/hsiaoyuhsien/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(288, 432,..., activation="relu")`
  # Remove the CWD from sys.path while we load stuff.


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 286, 430, 32)      320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 143, 215, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 141, 213, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 70, 106, 64)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 68, 104, 64)       36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 34, 52, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 113152)           

In [14]:
# fitting model, 測試1
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(train_data)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(train_data, train_label, batch_size=32),
                    steps_per_epoch=len(train_data) / 32, epochs=12)

Epoch 1/12
6/5 [================================] - 29s 5s/step - loss: 0.9656 - accuracy: 0.5562
Epoch 2/12
6/5 [================================] - 23s 4s/step - loss: 0.6972 - accuracy: 0.4944
Epoch 3/12
6/5 [================================] - 26s 4s/step - loss: 0.6906 - accuracy: 0.5337
Epoch 4/12


KeyboardInterrupt: 